In [1]:
%load_ext autoreload
%autoreload 2

import os 
import shutil
import numpy as n
from datetime import date
from matplotlib import pyplot as plt
import time
import napari

In [2]:
os.chdir('/home/ali/packages/s2p-lbm/')

from suite3d.job import Job
from suite3d import lbmio, utils, ui
from suite3d import tiff_utils as tfu
from suite3d import file_utils as flu

In [3]:
job_params = {    
    'planes' :         n.arange(0,15),
    'n_ch_tif' : 15,
    'n_init_files' :   6,
    'fuse_shift_override' : 6,
    'subtract_crosstalk' : False,
    'fix_fastZ' : 0,
}

In [4]:
subjects_dir =   '/mnt/zortex-subjects/'
expt_info = { 
    'subject':         'AH011',
    'date' :          '2024-02-02',
    'expnum' :         [1,4,6],}

tifs, si_params, exp_str = flu.find_exp(subjects_dir, **expt_info, verbose=True, get_si_params=True)
job_params['fs'] = si_params['vol_rate']

# # savedir = os.path.join('/mnt/md0/runs', expt_info['subject'], expt_info['date'])
# # os.makedirs(savedir, exist_ok=True)

Looking for AH011/2024-02-02/1 in /mnt/zortex-subjects/
Found
Looking for AH011/2024-02-02/4 in /mnt/zortex-subjects/
Found
Looking for AH011/2024-02-02/6 in /mnt/zortex-subjects/
Found


In [5]:
# Create the job
job = Job('/mnt/md0/runs',exp_str, tifs = tifs,params_path='/mnt/md0/runs/s3d-AH011_2024-02-02_1-4-6/params.npy',
          params=job_params, create=False, overwrite=True, verbosity = 3)
# job.update_root_path('/mnt/md0/runs')

   Found and loaded params from /mnt/md0/runs/s3d-AH011_2024-02-02_1-4-6/params.npy


In [6]:
summary = job.load_summary()

In [6]:
job.make_extension_dir('/mnt/a3/runs')

   Made new extension dir at /mnt/a3/runs/s3d-extension-AH011_2024-02-02_1-4-6


'/mnt/a3/runs/s3d-extension-AH011_2024-02-02_1-4-6'

In [6]:
# optional parameters for initialization
# load 1 file to initialize, and select 200 frames randomly to do initalization on
job.params['n_init_files'] = 6
job.params['init_n_frames'] = None
job.params['fuse_shift_override'] = 6

In [7]:
# %%time
# summary = job.run_init_pass()

In [8]:
# job.show_summary_plots()

In [9]:
summary = job.load_summary()

In [10]:
# # Pick the parameters you want to sweep, and enter them in the tuples.
# # It will do all combinations of parameters, so if you have many parameters it will be many combinations! 
# job.params['t_batch_size'] = 200
# job.params['edge_crop_npix'] = 10
# params_to_sweep = {
#     'intensity_thresh' : (0.1,),
#     'conv_filt_xy': (1.0,),
#     'npil_filt_xy': (20.0,),
#     'conv_filt_z' : (1.0,1.5,),
#     'npil_filt_z' : (1.0,),
#     'sdnorm_exp' : (1.0,),
#     'edge_crop_npix' : (0,20),
# }

# # sweep_summary = job.sweep_params(params_to_sweep, mov = None,testing_dir_tag='sweep-test',
# #                            do_vmap=True, all_combinations=True, n_test_iters = 1)
# # # use this to access older sweep results
# # sweep_summary = n.load(os.path.join(job.dirs['sweep-full'], 'sweep_summary.npy'),allow_pickle=True).item()
# # job.vis_vmap_sweep(sweep_summary)

In [11]:
mov = job.get_registered_movie()

In [12]:
job.params['t_batch_size'] = 400
job.params['voxel_size_um'] = (15, 2.5, 2.5)
job.params['npil_filt_type'] = 'unif'
job.params['npil_filt_xy_um'] = 70
job.params['npil_filt_z_um'] = 15.0
job.params['cell_filt_type'] = 'gaussian'
job.params['cell_filt_xy_um'] = 2.5
job.params['cell_filt_z_um'] = 22.5
job.params['edge_crop_npix'] = 10
job.params['fix_vmap_edge_planes'] = True
job.params['sdnorm_exp'] = 0.85
job.params['intensity_thresh'] = 0.1
job.params['n_proc'] = 32


In [13]:
# corrmap_new = job.calculate_corr_map(mov, save=True)

In [14]:
# results = job.load_corr_map_results()

In [15]:
job.verbosity = 2
job.params['detection_timebin'] = 2
job.params['detection_time_crop'] = (None,None)
job.params['max_pix'] = 250
job.params['peak_thresh'] = 0.1
job.params['patch_size_xy'] = (150,150)
job.params['n_proc_detect'] = 8
job.params['activity_thresh'] = 1.0
job.params['percentile'] = 99.0
job.params['extend_thresh'] = 0.15
job.params['max_iter'] = 10000 # maximum number of ROIs detected in given patch

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs


In [16]:

# job.segment_rois()

In [18]:
%%time
job.compute_npil_masks(job.dirs['rois'])

CPU times: user 39.2 s, sys: 3.77 s, total: 43 s
Wall time: 2min 12s


'/mnt/md0/runs/s3d-AH011_2024-02-02_1-4-6/rois'

In [19]:
traces = job.extract_and_deconvolve(stats_dir = job.dirs['rois'], mov=mov, crop=False)

   Updated main params file
   Movie shape: (15, 20711, 1060, 765)
11915
   Extracting 11915 valid cells, and saving cell flags to /mnt/md0/runs/s3d-AH011_2024-02-02_1-4-6/rois/iscell_extracted.npy
   Extracting activity
Will extract in 42 batches of 500
Saving intermediate results to /mnt/md0/runs/s3d-AH011_2024-02-02_1-4-6/rois
Extracting batch 0000 of 0042
Batch size: 22 GB
Extracting batch 0001 of 0042
Batch size: 22 GB
Extracting batch 0002 of 0042
Batch size: 22 GB
Extracting batch 0003 of 0042
Batch size: 22 GB
Extracting batch 0004 of 0042
Batch size: 22 GB
Extracting batch 0005 of 0042
Batch size: 22 GB
Extracting batch 0006 of 0042
Batch size: 22 GB
Extracting batch 0007 of 0042
Batch size: 22 GB
Extracting batch 0008 of 0042
Batch size: 22 GB
Extracting batch 0009 of 0042
Batch size: 22 GB
Extracting batch 0010 of 0042
Batch size: 22 GB
Extracting batch 0011 of 0042
Batch size: 22 GB
Extracting batch 0012 of 0042
Batch size: 22 GB
Extracting batch 0013 of 0042
Batch size: 22

In [9]:
job.export_results(export_path='/mnt/zeytin-f1/s3d-results/', result_dir_name='rois')

   Created dir /mnt/zeytin-f1/s3d-results/s3d-results-AH011_2024-02-02_1-4-6 to export results
      Loading from /mnt/md0/runs/s3d-AH011_2024-02-02_1-4-6/rois/stats_small.npy
      Loading from /mnt/md0/runs/s3d-AH011_2024-02-02_1-4-6/rois/info.npy
      Loading from /mnt/md0/runs/s3d-AH011_2024-02-02_1-4-6/rois/F.npy
      Loading from /mnt/md0/runs/s3d-AH011_2024-02-02_1-4-6/rois/spks.npy
      Loading from /mnt/md0/runs/s3d-AH011_2024-02-02_1-4-6/rois/Fneu.npy
      Loading from /mnt/md0/runs/s3d-AH011_2024-02-02_1-4-6/rois/iscell.npy
      Overwriting existing /mnt/zeytin-f1/s3d-results/s3d-results-AH011_2024-02-02_1-4-6/s3d-params.npy
      Overwriting existing /mnt/zeytin-f1/s3d-results/s3d-results-AH011_2024-02-02_1-4-6/frames.npy
      Overwriting existing /mnt/zeytin-f1/s3d-results/s3d-results-AH011_2024-02-02_1-4-6/stats.npy
      Saved stats.npy to /mnt/zeytin-f1/s3d-results/s3d-results-AH011_2024-02-02_1-4-6
      Overwriting existing /mnt/zeytin-f1/s3d-results/s3d-results

In [ ]:
from suite3d 